In [2]:
import psycopg2 as pg
import numpy as np
import scipy
import matplotlib as plt
%matplotlib inline

In [3]:
conn = pg.connect(database="weather", user="weather_ro", host="localhost")
cur = conn.cursor()

In [6]:
cur.execute("SET SESSION TIME ZONE UTC;")

In [8]:
cur.execute("SELECT count(*) FROM tblWeatherHistoric;")
print("{} rows found.".format(cur.fetchall()[0][0]))

346105 rows found.
